## Predict the auction sale price for a piece of heavy equipment to create a "blue book" for bulldozers.

The key fields are in train.csv are:

1. SalesID: the uniue identifier of the sale
MachineID: the unique identifier of a machine.A machine can be sold multiple times

2. saleprice: what the machine sold for at auction (only provided in train.csv)

3. saledate: the date of the sale

There are several fields towards the end of the file on the different options a machine can have.  The descriptions all start with "machine configuration" in the data dictionary.  Some product types do not have a particular option, so all the records for that option variable will be null for that product type.  Also, some sources do not provide good option and/or hours data.
The machine_appendix.csv file contains the correct year manufactured for a given machine along with the make, model, and product class details. There is one machine id for every machine in all the competition datasets (training, evaluation, etc.).

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

In [2]:
df_raw = pd.read_feather("data/bulldozer-train.feather")
#df = df_raw.copy()

/Users/ujjawalpathak/anaconda/lib/python2.7/site-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


In [3]:
df = df_raw.copy()

In [4]:
def sniff(df):
    with pd.option_context("display.max_colwidth", 20):
        info = pd.DataFrame()
        info['sample'] = df.iloc[0]
        info['data type'] = df.dtypes
        info['percent missing'] = df.isnull().sum()*100/len(df)
        return info.sort_values('data type')

In [5]:
sniff(df).head(10)

,sample,data type,percent missing
SalesID,1646770,int64,0.000000
SalePrice,9500,int64,0.000000
MachineID,1126363,int64,0.000000
ModelID,8434,int64,0.000000
datasource,132,int64,0.000000
YearMade,1974,int64,0.000000
auctioneerID,18,float64,5.174687
MachineHoursCurrentMeter,NaN,float64,64.717764
saledate,1989-01-17 00:00:00,datetime64[ns],0.000000
Coupler,None,object,46.826855


In [6]:
print(df['Tire_Size']).unique()

[None '14"' 'None or Unspecified' '20.5' '23.5' '26.5' '17.5' '29.5' '13"'
 '20.5"' '23.5"' '17.5"' '15.5' '15.5"' '7.0"' '23.1"' '10"' '10 inch']


In [7]:
basefeatures = ['SalesID', 'MachineID', 'ModelID',
                'datasource', 'YearMade',
                # some missing values but use anyway:
                'auctioneerID', 'MachineHoursCurrentMeter']

In [8]:
X,y = df[basefeatures], df['SalePrice']
X = X.fillna(0)

In [9]:
rf = RandomForestRegressor(n_estimators=50, n_jobs=-1, oob_score=True)
rf.fit(X,y)
score = rf.oob_score_
print(score)

0.7811504867916343


In [10]:
del df['SalesID']

In [11]:
del df['MachineID']

In [12]:
df['auctioneerID'] = df['auctioneerID'].astype(str)

In [13]:
print(df['auctioneerID'].unique())

['18.0' '99.0' '4.0' '1.0' '6.0' '24.0' '19.0' '10.0' '2.0' '23.0' '11.0'
 '15.0' '25.0' '16.0' '13.0' '14.0' '7.0' '21.0' '5.0' '12.0' '28.0'
 '20.0' '22.0' '9.0' '17.0' '3.0' '8.0' 'nan' '27.0' '26.0' '0.0']


In [14]:
print(df['ProductGroup']).unique()

['TTT' 'BL' 'MG' 'WL' 'TEX' 'SSL']


In [15]:
print(df['Drive_System']).unique()

[None 'Two Wheel Drive' 'No' 'Four Wheel Drive' 'All Wheel Drive']


In [16]:
from pandas.api.types import is_string_dtype, is_object_dtype

def normalize_string(df):
    for col in df.columns:
        if is_string_dtype(df[col]) or is_object_dtype(df[col]):
            df[col] = df[col].str.lower()
            df[col] = df[col].fillna(np.nan)
            df[col] = df[col].replace('none or unspecified', np.nan)
            df[col] = df[col].replace('none', np.nan)
            df[col] = df[col].replace('#name?', np.nan)
            df[col] = df[col].replace('', np.nan)

In [17]:
normalize_string(df)

In [18]:
print(df['Drive_System']).unique()

[nan 'two wheel drive' 'no' 'four wheel drive' 'all wheel drive']


In [19]:
def extract_sizes(df, colname):
    df[colname] = df[colname].str.extract(r'([0-9.]*)', expand=True)
    df[colname] = df[colname].replace('',np.nan)
    df[colname] = pd.to_numeric(df[colname])

In [20]:
extract_sizes(df, 'Tire_Size')
extract_sizes(df, 'Undercarriage_Pad_Width')

In [21]:
print(df['Tire_Size']).unique()

[ nan 14.  20.5 23.5 26.5 17.5 29.5 13.  15.5  7.  23.1 10. ]


In [24]:
from pandas.api.types import is_categorical_dtype

def string_to_cat(df):
    for col in df.columns:
        if is_string_dtype(df[col]):
            df[col]  = df[col].astype('category').cat_as_ordered()
            

def df_cat_to_catcode(df):
    for col in df.columns:
        if is_categorical_dtype(df[col]):
            df[col] = df[col].cat.codes + 1